In [1]:
%reload_ext autoreload
%autoreload 2
from fastai.basics import *

# Rossmann

## Data preparation / Feature engineering

In [2]:
! wget http://files.fast.ai/part2/lesson14/rossmann.tgz

--2019-03-21 11:43:40--  http://files.fast.ai/part2/lesson14/rossmann.tgz
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7730448 (7.4M) [text/plain]
Saving to: ‘rossmann.tgz’

rossmann.tgz        100%[===================>]   7.37M  13.1MB/s    in 0.6s    

2019-03-21 11:43:40 (13.1 MB/s) - ‘rossmann.tgz’ saved [7730448/7730448]



In [27]:
! mv rossmann.tgz data/.

In [10]:
! cd data/

> Why doesn't the magic "! cd" ever work?

In [28]:
! ls data/

bears  camvid-tiramisu	rossmann.tgz


In [30]:
! tar xzvf data/rossmann.tgz

googletrend.csv
sample_submission.csv
state_names.csv
store.csv
store_states.csv
test.csv
train.csv
weather.csv


In [39]:
! mkdir data/rossmann

In [41]:
! tar xzvf data/rossmann.tgz -C data/rossmann

googletrend.csv
sample_submission.csv
state_names.csv
store.csv
store_states.csv
test.csv
train.csv
weather.csv


In [43]:
! ls data/rossmann/

googletrend.csv        state_names.csv	store_states.csv  train.csv
sample_submission.csv  store.csv	test.csv	  weather.csv


In [57]:
PATH = Path('data/rossmann/')

In [64]:
table_names = ['train', 'store', 'store_states', 'state_names', 'googletrend', 'weather', 'test']
tables = [pd.read_csv(PATH/f'{fname}.csv', low_memory=False) for fname in table_names]
train, store, store_states, state_names, googletrend, weather, test = tables

In [65]:
train.dtypes

Store             int64
DayOfWeek         int64
Date             object
Sales             int64
Customers         int64
Open              int64
Promo             int64
StateHoliday     object
SchoolHoliday     int64
dtype: object

In [66]:
train.StateHoliday = train.StateHoliday != '0'
test.StateHoliday = test.StateHoliday != '0'

In [67]:
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, suffixes=('', suffix))

In [68]:
weather = join_df(weather, state_names, 'file', 'StateName')

In [69]:
googletrend.head()

,file,week,trend
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67


In [70]:
googletrend['Date'] = googletrend.week.str.split(' - ', expand=True)[0]
googletrend['State'] = googletrend.file.str.split('_', expand=True)[2]

In [71]:
googletrend.loc[googletrend.State=='NI', 'State'] = 'HB,NI'

In [72]:
googletrend.State.unique()

array(['SN', None, 'BY', 'TH', 'NW', 'BE', 'RP', 'BW', 'HB,NI', 'SH', 'HE', 'ST', 'HH', 'SL'], dtype=object)

In [73]:
def add_datepart(df, fldname, drop=True, time=False):
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64
    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 
            'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10**9
    if drop: df.drop(fldnamej, axis=1, inplace=True)

In [74]:
add_datepart(weather, "Date", drop=False)
add_datepart(googletrend, "Date", drop=False)
add_datepart(train, "Date", drop=False)
add_datepart(test, "Date", drop=False)

In [13]:
trend_de = googletrend[googletrend.file == 'Rossmann_DE']

In [14]:
store_states.columns

Index(['Store', 'State'], dtype='object')

In [15]:
store = join_df(store, store_states, 'Store')
assert any(store.State.isna()) == False

In [16]:
joined = join_df(train, store, 'Store')
joined_test = join_df(test, store, 'Store')
assert any(joined.State.isna()) == False
assert any(joined_test.State.isna()) == False

In [17]:
joined = join_df(joined, googletrend, ['State', 'Year', 'Week'])
joined_test = join_df(joined_test, googletrend, ['State', 'Year', 'Week'])
assert any(joined.trend.isna()) == False
assert any(joined_test.trend.isna()) == False

In [18]:
joined = joined.merge(trend_de, on=['Year', 'Week'], how='left', suffixes=('', '_DE'))
joined_test = joined_test.merge(trend_de, on=['Year', 'Week'], how='left', suffixes=('', '_DE'))
assert any(joined.trend_DE.isna()) == False
assert any(joined_test.trend_DE.isna()) == False

In [19]:
joined = join_df(joined, weather, ['State', 'Date'])
joined_test = join_df(joined_test, weather, ['State', 'Date'])
assert any(joined.Max_TemperatureC.isna()) == False
assert any(joined_test.Max_TemperatureC.isna()) == False

In [20]:
for df in (joined, joined_test):
    to_remove = set(c for c in df.columns if c.endswith('_y'))
    for n in to_remove: df.drop(n, axis=1, inplace=True)

In [23]:
for df in (joined, joined_test):
    df['CompetitionOpenSinceYear'] = df.CompetitionOpenSinceYear.fillna(1900).astype(np.int32)
    df['CompetitionOpenSinceMonth'] = df.CompetitionOpenSinceMonth.fillna(1).astype(np.int32)
    df['Promo2SinceYear'] = df.Promo2SinceYear.fillna(1900).astype(np.int32)
    df['Promo2SinceWeek'] = df.Promo2SinceWeek.fillna(1).astype(np.int32)

In [24]:
for df in (joined, joined_test):
    df["CompetitionOpenSince"] = pd.to_datetime(dict(year=df.CompetitionOpenSinceYear, 
                                                     month=df.CompetitionOpenSinceMonth, day=15))
    df["CompetitionDaysOpen"] = df.Date.subtract(df.CompetitionOpenSince).dt.days

In [25]:
for df in (joined,joined_test):
    df.loc[df.CompetitionDaysOpen<0, "CompetitionDaysOpen"] = 0
    df.loc[df.CompetitionOpenSinceYear<1990, "CompetitionDaysOpen"] = 0

In [26]:
for df in (joined,joined_test):
    df["CompetitionMonthsOpen"] = df["CompetitionDaysOpen"]//30
    df.loc[df.CompetitionMonthsOpen>24, "CompetitionMonthsOpen"] = 24

In [27]:
from isoweek import Week
for df in (joined,joined_test):
    df["Promo2Since"] = pd.to_datetime(df.apply(lambda x: Week(
        x.Promo2SinceYear, x.Promo2SinceWeek).monday(), axis=1).astype(pd.datetime))  # .astype(pd.datetime) is probably not needed.
    df["Promo2Days"] = df.Date.subtract(df["Promo2Since"]).dt.days

In [28]:
for df in (joined,joined_test):
    df.loc[df.Promo2Days<0, "Promo2Days"] = 0
    df.loc[df.Promo2SinceYear<1990, "Promo2Days"] = 0
    df["Promo2Weeks"] = df["Promo2Days"]//7
    df.loc[df.Promo2Weeks<0, "Promo2Weeks"] = 0
    df.loc[df.Promo2Weeks>25, "Promo2Weeks"] = 25

In [ ]:
joined.to_pickle(PATH/'joined')
joined_test.to_pickle(PATH/'joined_test')

In [5]:
(PATH).ls()

[PosixPath('data/rossmann/store.csv'),
 PosixPath('data/rossmann/state_names.csv'),
 PosixPath('data/rossmann/store_states.csv'),
 PosixPath('data/rossmann/train.csv'),
 PosixPath('data/rossmann/weather.csv'),
 PosixPath('data/rossmann/googletrend.csv'),
 PosixPath('data/rossmann/test.csv'),
 PosixPath('data/rossmann/sample_submission.csv'),
 PosixPath('data/rossmann/joined'),
 PosixPath('data/rossmann/joined_test')]

In [6]:
joined = pd.read_pickle(PATH/'joined')

In [11]:
joined_test = pd.read_pickle(PATH/'joined_test')

## Duration

In [114]:
columns = ['Date', 'Store', 'Promo', 'StateHoliday', 'SchoolHoliday']

In [115]:
df = train[columns].append(test[columns])

In [116]:
def get_elapsed(fld, pre):
    day1 = np.timedelta64(1, 'D')
    last_date, last_store = np.datetime64(), 0
    res = []
    for s, v, d in zip(df.Store.values, df[fld].values, df.Date.values):
        if s != last_store: last_store, last_date = s, np.datetime64()
        if v: last_date = d
        res.append((d - last_date).astype('timedelta64[D]') / day1)
    df[pre + fld] = res

In [117]:
fld = 'SchoolHoliday'
pre, df = 'After', df.sort_values(['Store', 'Date'])
get_elapsed(fld, pre)
pre, df = 'Before', df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, pre)

In [118]:
fld = 'StateHoliday'
pre, df = 'After', df.sort_values(['Store', 'Date'])
get_elapsed(fld, pre)
pre, df = 'Before', df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, pre)

In [119]:
%%time
fld = 'Promo'
pre, df = 'After', df.sort_values(['Store', 'Date'])
get_elapsed(fld, pre)
pre, df = 'Before', df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, pre)

CPU times: user 23.2 s, sys: 16 ms, total: 23.2 s
Wall time: 21.2 s


In [121]:
df = df.set_index('Date')

In [123]:
columns = ['SchoolHoliday', 'StateHoliday', 'Promo']

In [125]:
for o in ['Before', 'After']:
    for p in columns:
        a = o + p
        df[a] = df[a].fillna(0).astype(int)

In [173]:
bwd = df[['Store'] + columns].sort_index().groupby('Store').rolling(7, min_periods=1).sum()

In [161]:
fwd = df[['Store'] + columns].sort_index(ascending=False).groupby('Store').rolling(7, min_periods=1).sum()

In [174]:
bwd.drop('Store', axis=1, inplace=True)
bwd.reset_index(inplace=True)

In [176]:
fwd.drop('Store', axis=1, inplace=True)
fwd.reset_index(inplace=True)

In [179]:
df.reset_index(inplace=True)

In [181]:
df.columns

Index(['Date', 'Store', 'Promo', 'StateHoliday', 'SchoolHoliday',
       'AfterSchoolHoliday', 'BeforeSchoolHoliday', 'AfterStateHoliday',
       'BeforeStateHoliday', 'AfterPromo', 'BeforePromo'],
      dtype='object')

In [182]:
bwd.columns

Index(['Store', 'Date', 'SchoolHoliday', 'StateHoliday', 'Promo'], dtype='object')

In [183]:
fwd.columns

Index(['Store', 'Date', 'SchoolHoliday', 'StateHoliday', 'Promo'], dtype='object')

In [185]:
df = df.merge(bwd, left_on=['Date', 'Store'], right_on=['Date', 'Store'], how='left', suffixes=['', '_bw'])

In [186]:
%%time
df = df.merge(fwd, left_on=['Date', 'Store'], right_on=['Date', 'Store'], how='right', suffixes=['', '_fw'])

CPU times: user 1.05 s, sys: 72 ms, total: 1.12 s
Wall time: 300 ms


In [187]:
df.columns

Index(['Date', 'Store', 'Promo', 'StateHoliday', 'SchoolHoliday',
       'AfterSchoolHoliday', 'BeforeSchoolHoliday', 'AfterStateHoliday',
       'BeforeStateHoliday', 'AfterPromo', 'BeforePromo', 'SchoolHoliday_bw',
       'StateHoliday_bw', 'Promo_bw', 'SchoolHoliday_fw', 'StateHoliday_fw',
       'Promo_fw'],
      dtype='object')

In [189]:
df.drop(columns, axis=1, inplace=True)

In [191]:
%%time
df.to_pickle(PATH/'df')

CPU times: user 104 ms, sys: 252 ms, total: 356 ms
Wall time: 356 ms


In [196]:
joined = pd.read_pickle(PATH/'joined')
joined_test = pd.read_pickle(PATH/'joined_test')

In [197]:
joined.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Week', 'Day',
       'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start',
       'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start',
       'Elapsed', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'State', 'file',
       'week', 'trend', 'file_DE', 'week_DE', 'trend_DE', 'Date_DE',
       'State_DE', 'Month_DE', 'Day_DE', 'Dayofweek_DE', 'Dayofyear_DE',
       'Is_month_end_DE', 'Is_month_start_DE', 'Is_quarter_end_DE',
       'Is_quarter_start_DE', 'Is_year_end_DE', 'Is_year_start_DE',
       'Elapsed_DE', 'Max_TemperatureC', 'Mean_TemperatureC',
       'Min_TemperatureC', 'Dew_PointC', 'MeanDew_PointC', 'Min_DewpointC',
       'Max_Humidity', 'Mean_Humidity', 'Min_Humidity',
       'Max_Sea

In [202]:
joined = join_df(left=joined, right=df, left_on=['Store', 'Date'])
joined_test = join_df(left=joined_test, right=df, left_on=['Store', 'Date'])

In [207]:
joined = joined[joined.Sales != 0]

In [209]:
joined.reset_index(inplace=True)
joined_test.reset_index(inplace=True)

In [210]:
joined.to_pickle(PATH/'train_clean')
joined_test.to_pickle(PATH/'test_clean')

In [211]:
PATH.ls()

[PosixPath('data/rossmann/df'),
 PosixPath('data/rossmann/store.csv'),
 PosixPath('data/rossmann/state_names.csv'),
 PosixPath('data/rossmann/store_states.csv'),
 PosixPath('data/rossmann/train.csv'),
 PosixPath('data/rossmann/weather.csv'),
 PosixPath('data/rossmann/googletrend.csv'),
 PosixPath('data/rossmann/test.csv'),
 PosixPath('data/rossmann/sample_submission.csv'),
 PosixPath('data/rossmann/joined'),
 PosixPath('data/rossmann/joined_test'),
 PosixPath('data/rossmann/train_clean'),
 PosixPath('data/rossmann/test_clean')]